In [1]:
#This is the implementation of the prophet model on our dataset
import pandas as pd
from prophet import Prophet

# Load data
df = pd.read_csv("archive/Stocks/aapl.us.txt")

c:\Users\Patty\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
